In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import math
import plotly.express as px
import scipy.interpolate as interpolate
import time
import datetime
import ciso8601
import os
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))
import re

In [5]:
wheel_perim = 1.25 * 2 * np.pi 
mm_per_degree = wheel_perim / 360


In [6]:
ffolder = 'Z:\\Jason Gao\\Satiety_behavior_JG_041921\\050821\\20210508-154659 JGWT1\\'
raw_encoderdata = pd.read_csv(ffolder+'rotaryencoder_data.csv', delimiter = ',', names =["Type","PC_TIME","EVT_TIME",'VALUE'])
t = raw_encoderdata['PC_TIME'].iloc[0]
ts= ciso8601.parse_datetime(t)
t0_encoder= time.mktime(ts.timetuple())+ts.microsecond/1000000

pybpod_file = [f for f in os.listdir(ffolder) if f.endswith(".csv") and hasNumbers(f)]
pybpod_data = pd.read_csv(ffolder + pybpod_file[0], skiprows=6, delimiter = ';')
t = pybpod_data['PC-TIME'].iloc[0]
ts= ciso8601.parse_datetime(t)
t0_pybpod= time.mktime(ts.timetuple())+ts.microsecond/1000000

#Find the latency btw encoder and pybpod data, and rezero time
print(t0_encoder-t0_pybpod)
t0_latency = round(t0_encoder-t0_pybpod,3) #limit decimal to 3 decimal places
raw_encoderdata.EVT_TIME = raw_encoderdata.EVT_TIME - raw_encoderdata.EVT_TIME[0]
raw_encoderdata.EVT_TIME =np.ceil(raw_encoderdata.EVT_TIME/1000.0) 

#Find the angular displacement and convert to mm
raw_encoderdata.VALUE = raw_encoderdata.VALUE.diff() #Find angular differences
raw_encoderdata.loc[raw_encoderdata['VALUE'] > 1, 'VALUE'] = -0.9#find -180to180 data points and convert
raw_encoderdata.loc[raw_encoderdata['VALUE'] < -1, 'VALUE'] = 0.9#find -180to180 data points and convert
raw_encoderdata.VALUE = raw_encoderdata.VALUE * -mm_per_degree 

raw_encoderdata

2.152323007583618


,Type,PC_TIME,EVT_TIME,VALUE
0,P,2021-05-08 15:47:02.978,0.0,NaN
1,P,2021-05-08 15:47:03.139,1.0,0.019635
2,P,2021-05-08 15:47:03.152,1.0,0.019635
3,P,2021-05-08 15:47:03.161,1.0,0.019635
4,P,2021-05-08 15:47:03.175,1.0,0.019635
...,...,...,...,...
60725,P,2021-05-08 16:35:53.877,2931.0,0.019635
60726,P,2021-05-08 16:35:54.513,2932.0,0.019635
60727,P,2021-05-08 16:35:56.700,2934.0,0.019635
60728,P,2021-05-08 16:36:05.658,2943.0,0.019635


In [7]:
raw_encoderdata1000ms = raw_encoderdata.groupby(['EVT_TIME']).sum().reset_index()
# %store raw_encoderdata1000ms
raw_encoderdata1000ms

,EVT_TIME,VALUE
0,0.0,0.000000
1,1.0,1.138827
2,2.0,-0.157080
3,3.0,4.987278
4,4.0,1.237002
...,...,...
622,2931.0,0.039270
623,2932.0,0.019635
624,2934.0,0.019635
625,2943.0,0.019635


In [8]:
fig = px.line(x = raw_encoderdata1000ms.EVT_TIME, y= raw_encoderdata1000ms.VALUE)

In [9]:
f = interpolate.interp1d(raw_encoderdata1000ms.EVT_TIME, raw_encoderdata1000ms.VALUE)
speed1s = f(np.arange(raw_encoderdata1000ms.EVT_TIME.iloc[-1]))
speed1ms = f(np.arange(1,raw_encoderdata1000ms.EVT_TIME.iloc[-1],1/1000))
# print(np.arange(1,raw_encoderdata1000ms.EVT_TIME.iloc[-1],1/1000))
time_speed1s = np.arange(0,np.size(speed1s),1) + t0_latency
time_speed1ms = np.arange(0,np.size(speed1ms)/1000,1/1000) + t0_latency
%store speed1s
%store speed1ms
%store time_speed1s

Stored 'speed1s' (ndarray)
Stored 'speed1ms' (ndarray)
Stored 'time_speed1s' (ndarray)


In [81]:
np.arange(1,raw_encoderdata1000ms.EVT_TIME.iloc[-1],1/1000)

array([1.000000e+00, 1.001000e+00, 1.002000e+00, ..., 3.275997e+03,
       3.275998e+03, 3.275999e+03])

In [146]:
time_speed1ms = (np.arange(1,raw_encoderdata1000ms.EVT_TIME.iloc[-1],1/1000))

In [148]:
print(np.size(time_speed1ms))
time_speed1ms

3017000


array([1.000000e+00, 1.001000e+00, 1.002000e+00, ..., 3.017997e+03,
       3.017998e+03, 3.017999e+03])

In [143]:
[x * 0.1 for x in range(0, 10)]


[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]